In [1]:
# Load the uploaded force data file
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
# Load the uploaded force data file
file_path = r"C:\Users\opeye\Desktop\ALL GAIT DATA\SUBJECT 1\GP1_0.9_force.csv"
force_data = pd.read_csv(file_path)

# Display the first few rows
print(force_data.head())

       FP1_x       FP2_x      FP1_y      FP2_y       FP1_z       FP2_z
0  92.200729 -131.305817  10.403254  31.219845  142.717987  390.675629
1  90.718224 -130.603073  10.320055  31.523193  140.853226  394.306702
2  89.188599 -129.837616  10.222217  31.807592  139.063828  398.151276
3  87.615822 -129.014847  10.110712  32.074539  137.340393  402.192719
4  86.003807 -128.140030   9.986480  32.325405  135.674103  406.415070


In [2]:
# Extract vertical ground reaction forces
fp1_z = force_data['FP1_z']  # Left foot
fp2_z = force_data['FP2_z']  # Right foot

# Find heel strikes using peak detection on vertical force
# Only consider forces above 100 N to avoid noise
peaks_fp1, _ = find_peaks(fp1_z, height=100, distance=200)
peaks_fp2, _ = find_peaks(fp2_z, height=100, distance=200)


In [3]:
# STEP TIMING OF SUBJECT 1
step_times_left = pd.Series(peaks_fp1).diff().dropna() / 1000  # in seconds
step_times_left
import numpy as np

# Calculate step timing (in seconds) for both feet
step_times_left = np.diff(peaks_fp1) / 1000  # since sampling rate is 1000 Hz
step_times_right = np.diff(peaks_fp2) / 1000

# Calculate peak forces at heel strikes
peak_forces_left = fp1_z.iloc[peaks_fp1].values
peak_forces_right = fp2_z.iloc[peaks_fp2].values

# Calculate symmetry features (using length-limited versions to match lengths)
min_len = min(len(step_times_left), len(step_times_right), len(peak_forces_left), len(peak_forces_right))
features_df = pd.DataFrame({
    'step_time_left': step_times_left[:min_len],
    'step_time_right': step_times_right[:min_len],
    'peak_force_left': peak_forces_left[:min_len],
    'peak_force_right': peak_forces_right[:min_len],
    'step_time_diff': np.abs(step_times_left[:min_len] - step_times_right[:min_len]),
    'force_asymmetry': np.abs(peak_forces_left[:min_len] - peak_forces_right[:min_len])
})

In [4]:
import numpy as np

def extract_gait_phase_features(fpz, sampling_rate=1000, threshold=20):
    """
    Extract stance time, swing time, and impulse from vertical force (FPz) signal.
    
    Parameters:
        fpz (array-like): Vertical ground reaction force for one foot (e.g. FP1_z)
        sampling_rate (int): Sampling frequency in Hz (default 1000)
        threshold (float): Force threshold to detect contact (default 20 N)
    
    Returns:
        stance_times (np.array): Duration of stance phases in seconds
        swing_times (np.array): Duration of swing phases in seconds
        impulses (np.array): Force impulse during stance (Ns)
    """
    
    on_ground = fpz > threshold
    transitions = np.diff(on_ground.astype(int))

    # Detect start and end of stance phases
    stance_starts = np.where(transitions == 1)[0]
    stance_ends   = np.where(transitions == -1)[0]

    # Ensure equal number of starts and ends
    min_len = min(len(stance_starts), len(stance_ends))
    stance_starts = stance_starts[:min_len]
    stance_ends = stance_ends[:min_len]

    # Compute stance times
    stance_times = (stance_ends - stance_starts) / sampling_rate

    # Compute swing times (between consecutive stance phases)
    swing_times = (stance_starts[1:] - stance_ends[:-1]) / sampling_rate

    # Compute impulse for each stance phase
    impulses = []
    for start, end in zip(stance_starts, stance_ends):
        impulse = np.trapz(fpz[start:end], dx=1/sampling_rate)
        impulses.append(impulse)
    impulses = np.array(impulses)

    return stance_times, swing_times, impulses


In [5]:
# Left foot (FP1_z)
stance_left, swing_left, impulse_left = extract_gait_phase_features(fp1_z)

# Right foot (FP2_z)
stance_right, swing_right, impulse_right = extract_gait_phase_features(fp2_z)

In [9]:
min_len = min(len(features_df),
              len(stance_left), len(swing_left), len(impulse_left),
              len(stance_right), len(swing_right), len(impulse_right))

features_df = features_df.iloc[:min_len].copy()
features_df['stance_time_left'] = stance_left[:min_len]
features_df['swing_time_left'] = swing_left[:min_len]
features_df['impulse_left'] = impulse_left[:min_len]
features_df['stance_time_right'] = stance_right[:min_len]
features_df['swing_time_right'] = swing_right[:min_len]
features_df['impulse_right'] = impulse_right[:min_len]

In [11]:
features_df.head()

,step_time_left,step_time_right,peak_force_left,peak_force_right,step_time_diff,force_asymmetry,stance_time_left,swing_time_left,impulse_left,stance_time_right,swing_time_right,impulse_right
0,0.395,0.342,818.536133,774.417969,0.053,44.118164,-0.429,1.687,0.0,-0.444,1.740,0.0
1,0.888,0.942,823.087036,816.966248,0.054,6.120788,-0.426,1.759,0.0,-0.477,1.786,0.0
2,0.400,0.342,808.094299,782.941162,0.058,25.153137,-0.460,1.743,0.0,-0.452,1.752,0.0
3,0.945,0.980,842.157349,812.156799,0.035,30.000550,-0.451,1.753,0.0,-0.457,1.743,0.0
4,0.363,0.348,804.933350,783.784790,0.015,21.148560,-0.430,1.719,0.0,-0.451,1.713,0.0


In [12]:
# Load the uploaded marker data file
file_path = r"C:\Users\opeye\Desktop\ALL GAIT DATA\SUBJECT 1\GP1_0.9_marker.csv"
marker_data = pd.read_csv(file_path)

# Display the first few rows
print(marker_data.head())


    L_FCC_x   L_FM1_x   L_FM2_x   L_FM5_x   R_FCC_x   R_FM1_x   R_FM2_x  \
0  0.316924  0.466121  0.474338  0.446204  0.841898  1.054279  1.055588   
1  0.318552  0.464179  0.472360  0.443642  0.838462  1.050878  1.052208   
2  0.320408  0.462187  0.470309  0.440965  0.834794  1.047249  1.048597   
3  0.322510  0.460184  0.468218  0.438207  0.830912  1.043412  1.044775   
4  0.324876  0.458212  0.466126  0.435413  0.826842  1.039392  1.040765   

    R_FM5_x   L_FCC_y   L_FM1_y  ...   R_FM2_y   R_FM5_y   L_FCC_z   L_FM1_z  \
0  1.033168  0.586793  0.558632  ...  0.449196  0.397652  0.198899  0.063881   
1  1.029710  0.586653  0.558658  ...  0.449113  0.397564  0.203200  0.064915   
2  1.026020  0.586516  0.558700  ...  0.449028  0.397471  0.207801  0.066075   
3  1.022119  0.586386  0.558764  ...  0.448942  0.397373  0.212673  0.067369   
4  1.018030  0.586262  0.558852  ...  0.448857  0.397273  0.217781  0.068799   

    L_FM2_z   L_FM5_z   R_FCC_z   R_FM1_z   R_FM2_z   R_FM5_z  
0  0

In [13]:
                  import numpy as np

def extract_marker_features(marker_data, step_indices, side='R', sampling_rate=200):
    """
    Extract stride length, step width, vertical displacement, and foot velocity
    from marker data for one foot side ('L' or 'R').
    
    Parameters:
        marker_data (DataFrame): Marker positions (L_FCC_x, L_FCC_y, etc.)
        step_indices (list or array): Frame indices of heel strikes (e.g., from force data)
        side (str): 'L' for left or 'R' for right foot
        sampling_rate (int): Marker sampling rate in Hz (default 200)

    Returns:
        stride_lengths (np.array)
        step_widths (np.array)
        vertical_displacements (np.array)
        velocities (np.array)
    """
    x_col = f'{side}_FCC_x'
    y_col = f'{side}_FCC_y'
    z_col = f'{side}_FCC_z'

    stride_lengths = []
    step_widths = []
    vertical_displacements = []
    velocities = []

    for i in range(len(step_indices) - 1):
        i1, i2 = step_indices[i], step_indices[i + 1]

        # Stride Length (x-direction)
        dx = marker_data[x_col].iloc[i2] - marker_data[x_col].iloc[i1]
        stride_lengths.append(abs(dx))

        # Step Width (y-direction, between L and R at same time)
        if side == 'R':
            y_left = marker_data['L_FCC_y'].iloc[i1]
            y_right = marker_data['R_FCC_y'].iloc[i1]
        else:
            y_left = marker_data['L_FCC_y'].iloc[i1]
            y_right = marker_data['R_FCC_y'].iloc[i1]
        step_widths.append(abs(y_right - y_left))

        # Vertical Displacement (z-range between steps)
        z_range = marker_data[z_col].iloc[i1:i2].max() - marker_data[z_col].iloc[i1:i2].min()
        vertical_displacements.append(z_range)

        # Velocity (distance / time)
        distance = np.sqrt(
            (marker_data[x_col].iloc[i2] - marker_data[x_col].iloc[i1])**2 +
            (marker_data[y_col].iloc[i2] - marker_data[y_col].iloc[i1])**2 +
            (marker_data[z_col].iloc[i2] - marker_data[z_col].iloc[i1])**2
        )
        time = (i2 - i1) / sampling_rate
        velocities.append(distance / time if time > 0 else 0)

    return (np.array(stride_lengths),
            np.array(step_widths),
            np.array(vertical_displacements),
            np.array(velocities))
                  

In [14]:
# Convert force indices to marker indices
peaks_fp2_marker = (peaks_fp2 / 5).astype(int)
peaks_fp1_marker = (peaks_fp1 / 5).astype(int)


In [15]:
# For right foot
stride_R, width_R, zdisp_R, vel_R = extract_marker_features(marker_data, peaks_fp2_marker, side='R')

# For left foot
stride_L, width_L, zdisp_L, vel_L = extract_marker_features(marker_data, peaks_fp1_marker, side='L')


In [16]:
max_index = len(marker_data) - 1
peaks_fp2_marker = np.clip(peaks_fp2_marker, 0, max_index)
peaks_fp1_marker = np.clip(peaks_fp1_marker, 0, max_index)


In [17]:
min_len = min(len(features_df), len(stride_L), len(zdisp_L), len(vel_L), len(stride_R))

features_df = features_df.iloc[:min_len].copy()
features_df['stride_length_left'] = stride_L[:min_len]
features_df['stride_length_right'] = stride_R[:min_len]
features_df['step_width'] = width_R[:min_len]  # shared value
features_df['vertical_disp_left'] = zdisp_L[:min_len]
features_df['vertical_disp_right'] = zdisp_R[:min_len]
features_df['foot_velocity_left'] = vel_L[:min_len]
features_df['foot_velocity_right'] = vel_R[:min_len]
features_df['target'] = 0  # initially mark all as healthy
features_df.head()

,step_time_left,step_time_right,peak_force_left,peak_force_right,step_time_diff,force_asymmetry,stance_time_left,swing_time_left,impulse_left,stance_time_right,swing_time_right,impulse_right,stride_length_left,stride_length_right,step_width,vertical_disp_left,vertical_disp_right,foot_velocity_left,foot_velocity_right,target
0,0.395,0.342,818.536133,774.417969,0.053,44.118164,-0.429,1.687,0.0,-0.444,1.740,0.0,0.363086,0.316494,0.114326,0.027830,0.021371,0.922229,0.919685,0
1,0.888,0.942,823.087036,816.966248,0.054,6.120788,-0.426,1.759,0.0,-0.477,1.786,0.0,0.390280,0.361562,0.134784,0.232416,0.227643,0.439794,0.385463,0
2,0.400,0.342,808.094299,782.941162,0.058,25.153137,-0.460,1.743,0.0,-0.452,1.752,0.0,0.369506,0.315359,0.112726,0.019479,0.023671,0.925187,0.916916,0
3,0.945,0.980,842.157349,812.156799,0.035,30.000550,-0.451,1.753,0.0,-0.457,1.743,0.0,0.326542,0.259029,0.122132,0.228831,0.222682,0.346891,0.266097,0
4,0.363,0.348,804.933350,783.784790,0.015,21.148560,-0.430,1.719,0.0,-0.451,1.713,0.0,0.327821,0.310018,0.101405,0.042446,0.033500,0.906222,0.904318,0


In [18]:
len(features_df)

45

# Modify Their Feature Values (Simulate Stroke Patterns)


In [19]:
# generating synthetic anomalies, you would similarly pick some normal rows and tweak features,
import numpy as np

# Define percentage of rows to simulate
n_anomalies = int(0.3 * len(features_df))

# Select random rows to simulate
np.random.seed(42)
anomaly_indices = np.random.choice(features_df.index, n_anomalies, replace=False)

# Increase step time and step time difference
features_df.loc[anomaly_indices, 'step_time_left'] *= 1.4
features_df.loc[anomaly_indices, 'step_time_right'] *= 1.3
features_df.loc[anomaly_indices, 'step_time_diff'] *= 2.0

# Boost force asymmetry
features_df.loc[anomaly_indices, 'force_asymmetry'] *= 1.5

# Reduce stride length and vertical displacement
features_df.loc[anomaly_indices, 'stride_length_left'] *= 0.7
features_df.loc[anomaly_indices, 'stride_length_right'] *= 0.75
features_df.loc[anomaly_indices, 'vertical_disp_left'] *= 0.6
features_df.loc[anomaly_indices, 'vertical_disp_right'] *= 0.6

# Reduce foot velocity (dragging foot)
features_df.loc[anomaly_indices, 'foot_velocity_left'] *= 0.7
features_df.loc[anomaly_indices, 'foot_velocity_right'] *= 0.7


In [20]:
features_df.loc[anomaly_indices, 'target'] = 1
features_df.head()

,step_time_left,step_time_right,peak_force_left,peak_force_right,step_time_diff,force_asymmetry,stance_time_left,swing_time_left,impulse_left,stance_time_right,swing_time_right,impulse_right,stride_length_left,stride_length_right,step_width,vertical_disp_left,vertical_disp_right,foot_velocity_left,foot_velocity_right,target
0,0.3950,0.3420,818.536133,774.417969,0.053,44.118164,-0.429,1.687,0.0,-0.444,1.740,0.0,0.363086,0.316494,0.114326,0.027830,0.021371,0.922229,0.919685,0
1,0.8880,0.9420,823.087036,816.966248,0.054,6.120788,-0.426,1.759,0.0,-0.477,1.786,0.0,0.390280,0.361562,0.134784,0.232416,0.227643,0.439794,0.385463,0
2,0.4000,0.3420,808.094299,782.941162,0.058,25.153137,-0.460,1.743,0.0,-0.452,1.752,0.0,0.369506,0.315359,0.112726,0.019479,0.023671,0.925187,0.916916,0
3,1.3230,1.2740,842.157349,812.156799,0.070,45.000825,-0.451,1.753,0.0,-0.457,1.743,0.0,0.228579,0.194272,0.122132,0.137299,0.133609,0.242824,0.186268,1
4,0.5082,0.4524,804.933350,783.784790,0.030,31.722840,-0.430,1.719,0.0,-0.451,1.713,0.0,0.229475,0.232513,0.101405,0.025468,0.020100,0.634355,0.633023,1


In [25]:
# Save to CSV
output_file = 'SUBJECT 1 AT 0.9.csv'
features_df.to_csv(output_file, index=False)

output_file


'SUBJECT 1 AT 0.9 csv'